Hello World!
This notebook describes the effort to cleanup and cluster a Habits dataset.

de-duplicate : based on email i'd
               
Clustering the Data:
two methods - 

A) cluster the data  and see how many clusters are there: used **MeanShift method**


B) three bins : "signed up not used" aka less than 2 days, "completed not beyond 1 week" aka less than 8 days, "everyone else"

email capaign to Ressurrect users
Time period = April 30 to May 30

# Looking around the data set

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
# Import the required modules
import pandas as pd
import numpy as np
import scipy as sp

In [3]:
# simple function to read in the user data file.
# the argument parse_dates takes in a list of colums, which are to be parsed as date format
user_data_raw_csv = pd.read_csv("/home/eyebell/local_bin/janacare/janCC/datasets/Habits-Data_upto-7th-May.csv",\
                            parse_dates = [-3, -2, -1])

In [4]:
# import the pyexcel module
#import pyexcel as pe
#from pyexcel.ext import xls

# load the file
#records = pe.get_records(file_name="/home/eyebell/local_bin/janacare/datasets/Habits-Data_upto-7th-May.xls")
#len(records)
#for record in records:
    #print record

In [5]:
# data metrics
user_data_raw_csv.shape # Rows , colums


(53239, 9)

In [6]:
# data metrics
user_data_raw_csv.dtypes # data type of colums

user_id                   int64
first_name               object
last_name                object
username                 object
email                    object
phone_number             object
date_joined      datetime64[ns]
first_login      datetime64[ns]
last_activity    datetime64[ns]
dtype: object

In [7]:
user_data_to_clean = user_data_raw_csv.copy()

In [8]:
# Some basic statistical information on the data
#user_data_to_clean.describe()

# Data Clean up

In the last section of looking around, I saw that a lot of rows do not have any values or have garbage values(see first row of the table above).
This can cause errors when computing anything using the values in these rows, hence a clean up is required.

If a the coulums *last_activity* and *first_login* are empty then drop the corresponding row !

In [9]:
# Lets check the health of the data set
user_data_to_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53239 entries, 0 to 53238
Data columns (total 9 columns):
user_id          53239 non-null int64
first_name       53108 non-null object
last_name        5360 non-null object
username         53239 non-null object
email            53089 non-null object
phone_number     53094 non-null object
date_joined      53239 non-null datetime64[ns]
first_login      50773 non-null datetime64[ns]
last_activity    44903 non-null datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(5)
memory usage: 3.7+ MB


As is visible from the last column (*age_on_platform*) data type, Pandas is not recognising it as date type format. 
This will make things difficult, so I delete this particular column and add a new one.
Since the data in *age_on_platform* can be recreated by doing *age_on_platform* = *last_activity* - *first_login* 

But on eyeballing I noticed some, cells of column *first_login* have greater value than corresponding cell of *last_activity*. These cells need to be swapped, since its not possible to have *first_login* > *last_activity*
Finally the columns *first_login*, *last_activity* have missing values, as evident from above table. Since this is time data, that in my opinion should not be imputed, we will drop/delete the columns.

In [10]:
# Run a loop through the data frame and check each row for this anamoly, if found drop,
# this is being done ONLY for selected columns

import datetime

swapped_count = 0
first_login_count = 0
last_activity_count = 0
email_count = 0
userid_count = 0

for index, row in user_data_to_clean.iterrows():        
        if row.last_activity == pd.NaT or row.last_activity != row.last_activity:
            last_activity_count = last_activity_count + 1
            #print row.last_activity
            user_data_to_clean.drop(index, inplace=True)

        elif row.first_login > row.last_activity:
            user_data_to_clean.drop(index, inplace=True)
            swapped_count = swapped_count + 1

        elif row.first_login != row.first_login or row.first_login == pd.NaT:
            user_data_to_clean.drop(index, inplace=True)
            first_login_count = first_login_count + 1

        elif row.email != row.email: #or row.email == '' or row.email == ' ':
            user_data_to_clean.drop(index, inplace=True)
            email_count = email_count + 1

        elif row.user_id != row.user_id:
            user_data_to_clean.drop(index, inplace=True)
            userid_count = userid_count + 1

print "last_activity_count=%d\tswapped_count=%d\tfirst_login_count=%d\temail_count=%d\tuserid_count=%d" \
% (last_activity_count, swapped_count, first_login_count, email_count, userid_count)

last_activity_count=8336	swapped_count=922	first_login_count=170	email_count=2	userid_count=0


In [11]:
user_data_to_clean.shape

(43809, 9)

In [12]:
# Create new column 'age_on_platform' which has the corresponding value in date type format
user_data_to_clean["age_on_platform"] = user_data_to_clean["last_activity"] - user_data_to_clean["first_login"]

In [13]:
user_data_to_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43809 entries, 0 to 53238
Data columns (total 10 columns):
user_id            43809 non-null int64
first_name         43806 non-null object
last_name          2385 non-null object
username           43809 non-null object
email              43809 non-null object
phone_number       43801 non-null object
date_joined        43809 non-null datetime64[ns]
first_login        43809 non-null datetime64[ns]
last_activity      43809 non-null datetime64[ns]
age_on_platform    43809 non-null timedelta64[ns]
dtypes: datetime64[ns](3), int64(1), object(5), timedelta64[ns](1)
memory usage: 3.7+ MB


#### Validate if email i'd is correctly formatted

In [14]:
from validate_email import validate_email

email_count_invalid = 0
for index, row in user_data_to_clean.iterrows():        
        if not validate_email(row.email):
            #print row.email
            user_data_to_clean.drop(index, inplace=True)
            email_count_invalid = email_count_invalid + 1

print "Number of email-id invalid: %d" % (email_count_invalid)


Number of email-id invalid: 49


In [15]:
# Check the result of last operation 
user_data_to_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43760 entries, 0 to 53238
Data columns (total 10 columns):
user_id            43760 non-null int64
first_name         43757 non-null object
last_name          2381 non-null object
username           43760 non-null object
email              43760 non-null object
phone_number       43752 non-null object
date_joined        43760 non-null datetime64[ns]
first_login        43760 non-null datetime64[ns]
last_activity      43760 non-null datetime64[ns]
age_on_platform    43760 non-null timedelta64[ns]
dtypes: datetime64[ns](3), int64(1), object(5), timedelta64[ns](1)
memory usage: 3.7+ MB


### Remove duplicates

In [16]:
user_data_to_deDuplicate = user_data_to_clean.copy()

In [17]:
user_data_deDuplicateD = user_data_to_deDuplicate.loc[~user_data_to_deDuplicate.email.str.strip().duplicated()]
len(user_data_deDuplicateD)

40495

In [18]:
user_data_deDuplicateD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40495 entries, 0 to 53238
Data columns (total 10 columns):
user_id            40495 non-null int64
first_name         40492 non-null object
last_name          2111 non-null object
username           40495 non-null object
email              40495 non-null object
phone_number       40487 non-null object
date_joined        40495 non-null datetime64[ns]
first_login        40495 non-null datetime64[ns]
last_activity      40495 non-null datetime64[ns]
age_on_platform    40495 non-null timedelta64[ns]
dtypes: datetime64[ns](3), int64(1), object(5), timedelta64[ns](1)
memory usage: 3.4+ MB


## Clustering

In [44]:
# Now its time to convert the timedelta64 column named age_on_platform
def convert_timedelta64_to_sec(td64):
    ts = (td64 / np.timedelta64(1, 's'))
    return ts

user_data_deDuplicateD_timedelta64_converted = user_data_deDuplicateD.copy()
temp_copy = user_data_deDuplicateD.copy()
user_data_deDuplicateD_timedelta64_converted.drop("age_on_platform", 1)
user_data_deDuplicateD_timedelta64_converted['age_on_platform'] = temp_copy['age_on_platform'].apply(convert_timedelta64_to_sec)


In [45]:
user_data_deDuplicateD_timedelta64_converted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40495 entries, 0 to 53238
Data columns (total 10 columns):
user_id            40495 non-null int64
first_name         40492 non-null object
last_name          2111 non-null object
username           40495 non-null object
email              40495 non-null object
phone_number       40487 non-null object
date_joined        40495 non-null datetime64[ns]
first_login        40495 non-null datetime64[ns]
last_activity      40495 non-null datetime64[ns]
age_on_platform    40495 non-null float64
dtypes: datetime64[ns](3), float64(1), int64(1), object(5)
memory usage: 3.4+ MB


In [20]:
# Clustering using Mean shift
'''
from sklearn.cluster import MeanShift, estimate_bandwidth

#x = [1,1,5,6,1,5,10,22,23,23,50,51,51,52,100,112,130,500,512,600,12000,12230]
x = pd.Series(user_data_deDuplicateD_timedelta64_converted['age_on_platform'])

X = np.array(zip(x,np.zeros(len(x))), dtype=np.int)
bandwidth = estimate_bandwidth(X, quantile=0.2)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

for k in range(n_clusters_):
    my_members = labels == k
    print "cluster {0} : lenght = {1}".format(k, len(X[my_members, 0]))
    #print "cluster {0}: {1}".format(k, X[my_members, 0])
    cluster_sorted = sorted(X[my_members, 0])
    print "cluster {0} : Max = {2} days & Min {1} days".format(k, cluster_sorted[0]*1.15741e-5, cluster_sorted[-1]*1.15741e-5)
'''

cluster 0 : lenght = 36269
cluster 0 : Max = 111.888815445 days & Min 0.0 days
cluster 1 : lenght = 3891
cluster 1 : Max = 250.97137236 days & Min 111.934197491 days
cluster 2 : lenght = 122
cluster 2 : Max = 306.512584735 days & Min 251.177379766 days
cluster 3 : lenght = 67
cluster 3 : Max = 353.589646206 days & Min 307.752969458 days
cluster 4 : lenght = 63
cluster 4 : Max = 419.562120373 days & Min 355.577729363 days
cluster 5 : lenght = 26
cluster 5 : Max = 474.295703624 days & Min 425.044192838 days
cluster 6 : lenght = 14
cluster 6 : Max = 504.780227927 days & Min 478.80180168 days
cluster 7 : lenght = 7
cluster 7 : Max = 530.619521918 days & Min 508.479449177 days
cluster 8 : lenght = 11
cluster 8 : Max = 631.129585023 days & Min 579.371459826 days
cluster 9 : lenght = 7
cluster 9 : Max = 570.795179041 days & Min 538.63842877 days
cluster 10 : lenght = 5
cluster 10 : Max = 818.060200506 days & Min 797.947215638 days
cluster 11 : lenght = 5
cluster 11 : Max = 777.25270169 days &

In [21]:
# Clustering using Kmeans, not working
'''
y = [1,1,5,6,1,5,10,22,23,23,50,51,51,52,100,112,130,500,512,600,12000,12230]
y_float = map(float, y)
x = range(len(y))
x_float = map(float, x)

m = np.matrix([x_float, y_float]).transpose()


from scipy.cluster.vq import kmeans
kclust = kmeans(m, 5)

kclust[0][:, 0]

assigned_clusters = [abs(cluster_indices - e).argmin() for e in x]
'''

'\ny = [1,1,5,6,1,5,10,22,23,23,50,51,51,52,100,112,130,500,512,600,12000,12230]\ny_float = map(float, y)\nx = range(len(y))\nx_float = map(float, x)\n\nm = np.matrix([x_float, y_float]).transpose()\n\n\nfrom scipy.cluster.vq import kmeans\nkclust = kmeans(m, 5)\n\nkclust[0][:, 0]\n\nassigned_clusters = [abs(cluster_indices - e).argmin() for e in x]\n'

### Binning based on **age_on_platform** 
bin1 = 0 to 1 days

bin2 = 2 to 8 days

bin3 = 8 days and beyond

In [48]:
#user_data_binned = user_data_deDuplicateD.copy()
user_data_binned = user_data_deDuplicateD_timedelta64_converted.copy()
                   #user_data_deDuplicateD_timedelta64_converted
convert_sec_to_hr = lambda x: x/3600
user_data_binned["age_on_platform"] = user_data_binned['age_on_platform'].map(convert_sec_to_hr)

for index, row in user_data_deDuplicateD_timedelta64_converted.iterrows():
    if row["age_on_platform"] < 25:
        user_data_binned.set_value(index, 'bin', 1)
        
    elif row["age_on_platform"] >= 25 and row["age_on_platform"] < 49:
        user_data_binned.set_value(index, 'bin', 2)    
        
    elif row["age_on_platform"] >= 49 and row["age_on_platform"] < 169:
        user_data_binned.set_value(index, 'bin', 3)
        
    elif row["age_on_platform"] >=169 and row["age_on_platform"] < 337:
        user_data_binned.set_value(index, 'bin', 4)
        
    elif row["age_on_platform"] >=337 and row["age_on_platform"] < 505:
        user_data_binned.set_value(index, 'bin', 5)
        
    elif row["age_on_platform"] >=505 and row["age_on_platform"] < 673:
        user_data_binned.set_value(index, 'bin', 6)
        
    elif row["age_on_platform"] >=673 and row["age_on_platform"] < 1009:
        user_data_binned.set_value(index, 'bin', 7)
        
    elif row["age_on_platform"] >=1009 and row["age_on_platform"] < 1345:
        user_data_binned.set_value(index, 'bin', 8)
        
    

In [50]:
user_data_binned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40495 entries, 0 to 53238
Data columns (total 11 columns):
user_id            40495 non-null int64
first_name         40492 non-null object
last_name          2111 non-null object
username           40495 non-null object
email              40495 non-null object
phone_number       40487 non-null object
date_joined        40495 non-null datetime64[ns]
first_login        40495 non-null datetime64[ns]
last_activity      40495 non-null datetime64[ns]
age_on_platform    40495 non-null float64
bin                7333 non-null float64
dtypes: datetime64[ns](3), float64(2), int64(1), object(5)
memory usage: 3.7+ MB


In [66]:
print "Number of users with age_on_platform equal to 1 day or less aka 0 day = %d" % len(user_data_binned[user_data_binned.bin == 1])

Number of users with age_on_platform equal to 1 day or less aka 0 day = 1212


In [60]:
print "Number of users with age_on_platform between 1 and 2 days = %d" % len(user_data_binned[user_data_binned.bin == 2])

Number of users with age_on_platform between 1 and 2 days = 570


In [61]:
print "Number of users with age_on_platform greater than or equal to 2 complete days and less than 1 week = %d" % len(user_data_binned[user_data_binned.bin == 3])

Number of users with age_on_platform greater than or equal to 2 complete days and less than 1 week = 2172


In [62]:
print "Number of users with age_on_platform between 2nd week = %d" % len(user_data_binned[user_data_binned.bin == 4])

Number of users with age_on_platform between 2nd week = 1325


In [63]:
print "Number of users with age_on_platform between 3rd weeks = %d" % len(user_data_binned[user_data_binned.bin == 5])

Number of users with age_on_platform between 3rd weeks = 774


In [64]:
print "Number of users with age_on_platform between 4th weeks = %d" % len(user_data_binned[user_data_binned.bin == 6])

Number of users with age_on_platform between 4th weeks = 501


In [65]:
print "Number of users with age_on_platform greater than or equal to 4 weeks and less than 6 weeks = %d" % len(user_data_binned[user_data_binned.bin == 7])

Number of users with age_on_platform greater than or equal to 4 weeks and less than 6 weeks = 493


In [58]:
print "Number of users with age_on_platform greater than or equal to 6 weeks and less than 8 weeks = %d" % len(user_data_binned[user_data_binned.bin == 8])

Number of users with age_on_platform greater than or equal to 6 weeks and less than 8 weeks = 286


In [26]:
user_data_binned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40495 entries, 0 to 53238
Data columns (total 11 columns):
user_id            40495 non-null int64
first_name         40492 non-null object
last_name          2111 non-null object
username           40495 non-null object
email              40495 non-null object
phone_number       40487 non-null object
date_joined        40495 non-null datetime64[ns]
first_login        40495 non-null datetime64[ns]
last_activity      40495 non-null datetime64[ns]
age_on_platform    40495 non-null timedelta64[ns]
bin                40495 non-null float64
dtypes: datetime64[ns](3), float64(1), int64(1), object(5), timedelta64[ns](1)
memory usage: 3.7+ MB


In [27]:
# Save dataframe with binned values as CSV
user_data_binned.to_csv('user_data_binned.csv')

email capaign to Ressurrect users
Time period = April 30 to May 30

